In [2]:
import pandas as pd
# Importing all the necessary packages
import pandas as pd
from pandas.errors import PerformanceWarning
import matplotlib.pyplot as plt
import numpy as np
from hmmlearn.hmm import GaussianHMM
from scipy.stats import norm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from prophet import Prophet
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA

# ARIMA and Prophet both use .T which causes this annoying warning, so I'm hiding it 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=PerformanceWarning)
warnings.filterwarnings("ignore", message="No frequency information was provided, so inferred frequency 30min will be used.")
def metrics(y_test, y_pred):
    """
    Calculate and display error metrics for model evaluation.

    This function computes standard error metrics, including Mean Absolute Error (MAE), Mean Squared Error (MSE),
    Root Mean Squared Error (RMSE), and R-squared (R2), to evaluate the accuracy of model predictions.

    Parameters:
    - y_test (Series or array-like): The true values for the target variable in the test set.
    - y_pred (Series or array-like): The predicted values for the target variable in the test set.

    Returns:
    - None
    """
    # Calculate error metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Print error metrics
    print("Mean Absolute Error (MAE):", mae)
    print("Mean Squared Error (MSE):", mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("R-squared (R2):", r2)
    return rmse


In [15]:
answers = pd.read_excel("datathon_answers.xlsx")
merged = pd.read_csv("merged_df.csv")
answers["System_Price"].mean()
merged["System_Price"].mean()

87.63404999154834

In [17]:
merged[["Day_Ahead_Price", "NIV_Outturn"]].tail(48).reset_index(drop="index")

,Day_Ahead_Price,NIV_Outturn
0,10.00,400.09
1,14.00,259.82
2,5.00,-142.02
3,2.35,8.47
4,11.36,-347.94
5,9.60,124.90
6,10.60,448.31
7,13.40,353.56
8,32.00,432.78
9,46.20,238.41


In [12]:
# Select the last 48 rows of specified columns from merged and reset the index
merged_tail = merged.tail(48).reset_index(drop=True)
merged_tail["SPA"] = answers["System_Price"]
merged_tail["NIV"] = answers["NIV_Outturn"]

for col in merged:
    if col != "Datetime":
        try:
            metrics(merged_tail["SPA"], merged_tail[col])
        except ValueError:
            pass

Mean Absolute Error (MAE): 34.81520833333334
Mean Squared Error (MSE): 2019.67408125
Root Mean Squared Error (RMSE): 44.940784163719265
R-squared (R2): -1.6633742309609776
Mean Absolute Error (MAE): 353.32312500000006
Mean Squared Error (MSE): 203924.2818979167
Root Mean Squared Error (RMSE): 451.5797624981845
R-squared (R2): -267.9179816270088
Mean Absolute Error (MAE): 1255.0170833333334
Mean Squared Error (MSE): 2178637.6682999996
Root Mean Squared Error (RMSE): 1476.0208901976962
R-squared (R2): -2872.0018760055964
Mean Absolute Error (MAE): 1074.5047916666667
Mean Squared Error (MSE): 1752279.0708104167
Root Mean Squared Error (RMSE): 1323.7367830540998
R-squared (R2): -2309.7564561903296
Mean Absolute Error (MAE): 2012.8708333333334
Mean Squared Error (MSE): 4262133.417320834
Root Mean Squared Error (RMSE): 2064.493501399516
R-squared (R2): -5619.538688888004
Mean Absolute Error (MAE): 91.53333333333335
Mean Squared Error (MSE): 9136.665154166667
Root Mean Squared Error (RMSE): 9